## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

### Radiology Entity Detection, Assertion Status Assignment, and Relation Extraction Pipeline

Overview:

The Radiology Entity Detection, Assertion Status Assignment, and Relation Extraction Pipeline is a comprehensive solution designed to enhance the analysis of radiology documents. Its primary focus lies in identifying crucial entities within radiology texts, assigning assertion statuses to these entities, and extracting relevant relations among them.


- **Model**: `en.explain_doc.clinical_radiology.pipeline`
- **Model Description**: A pipeline for detecting posology entities with the `ner_radiology` NER model, assigning their assertion status with `assertion_dl_radiology` model, and extracting relations between posology-related terminology with `re_test_problem_finding` relation extraction model.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "en-explain-doc-clinical-radiology-pipeline"

content_type = "application/json"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"

### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

---------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Perform real-time inference

In [6]:
import json
import pandas as pd
import os
import boto3


# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def process_data_and_invoke_realtime_endpoint(data_dicts):
    for data_dict in data_dicts:
        json_input_data = json.dumps(data_dict)    
        i = 1
        input_file_name = f'inputs/real-time/input{i}.json'
        output_file_name = f'outputs/real-time/out{i}.out'

        while os.path.exists(input_file_name) or os.path.exists(output_file_name):
            i += 1
            input_file_name = f'inputs/real-time/input{i}.json'
            output_file_name = f'outputs/real-time/out{i}.out'

        os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
        os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

        with open(input_file_name, 'w') as f:
            f.write(json_input_data)

        s3_client.put_object(Bucket=s3_bucket, Key=f"{model_name}/validation-input-json/real-time/{os.path.basename(input_file_name)}", Body=bytes(json_input_data.encode('UTF-8')))

        response = sm_runtime.invoke_endpoint(
            EndpointName=model_name,
            ContentType=content_type,
            Accept="application/json",
            Body=json_input_data,
        )

        # Process response
        response_data = json.loads(response["Body"].read().decode("utf-8"))

        df_ner = pd.DataFrame(response_data['ner_predictions'])
        df_assertion = pd.DataFrame(response_data['assertion_predictions'])
        df_relation = pd.DataFrame(response_data['relation_predictions'])

        display(df_ner, df_assertion, df_relation)

        # Save response data to file
        with open(output_file_name, 'w') as f_out:
            json.dump(response_data, f_out, indent=4)


### Initial setup

In [7]:
docs = [
        """Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality  is discernible.""",

        """Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma."""
]

sample_text = """ Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new.  The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine"""

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [8]:
data_dicts = [
    {
        "text": sample_text
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,document,ner_chunk,begin,end,ner_label,confidence
0,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,Cardiac size,1,12,ImagingFindings,0.8463
1,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,left,41,44,Direction,0.9433
2,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,pleural,46,52,BodyPart,0.5032
3,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,effusion,54,61,ImagingFindings,0.9499
4,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,right,77,81,Direction,0.9554
5,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,effusion,83,90,ImagingFindings,0.9442
6,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,upper,105,109,Direction,0.7371
7,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,lungs,111,115,BodyPart,0.522
8,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,clear,121,125,ImagingFindings,0.7488
9,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,Right,128,132,Direction,0.9839


,document,ner_chunk,begin,end,ner_label,confidence,assertion
0,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,Cardiac size,1,12,ImagingFindings,0.8463,Confirmed
1,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,effusion,83,90,ImagingFindings,0.9442,Confirmed
2,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,clear,121,125,ImagingFindings,0.7488,Confirmed
3,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,pneumothorax,196,207,ImagingFindings,0.9832,Negative
4,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,mild degenerative changes,220,244,ImagingFindings,0.5607,Confirmed


,document,relation,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,confidence
0,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,not_related,upper lungs,105,115,BodyPart,clear,121,125,ImagingFindings,0.947544
1,Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine,is_related,mild degenerative changes,220,244,ImagingFindings,thoracic spine,253,266,BodyPart,0.99999964


#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [9]:
data_dicts = [
    {
        "text": docs
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,document,ner_chunk,begin,end,ner_label,confidence
0,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",bilateral foot,69,82,BodyPart,0.615
1,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",pain,84,87,Symptom,0.7286
2,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",traumatic,165,173,Disease_Syndrome_Disorder,0.7773
3,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",arthritic,176,184,Disease_Syndrome_Disorder,0.6663
4,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",developmental abnormality,190,214,ImagingFindings,0.4231
5,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",Bilateral breast,0,15,BodyPart,0.945
6,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ultrasound,17,26,Imaging_Test,0.6734
7,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ovoid mass,78,87,ImagingFindings,0.6095
8,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",0.5 x 0.5 x 0.4,113,127,Measurements,0.9815801
9,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",cm,129,130,Units,0.9696


,document,ner_chunk,begin,end,ner_label,confidence,assertion
0,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",traumatic,165,173,Disease_Syndrome_Disorder,0.7773,Negative
1,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",arthritic,176,184,Disease_Syndrome_Disorder,0.6663,Negative
2,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",developmental abnormality,190,214,ImagingFindings,0.4231,Negative
3,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ovoid mass,78,87,ImagingFindings,0.6095,Confirmed
4,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",mass,210,213,ImagingFindings,0.9711,Confirmed
5,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",isoechoic echotexture,228,248,ImagingFindings,0.80104995,Confirmed
6,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",internal color flow,294,312,ImagingFindings,0.47723332,Negative
7,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",benign fibrous tissue,334,354,ImagingFindings,0.5240666,Suspected
8,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",lipoma,361,366,Disease_Syndrome_Disorder,0.6081,Suspected


,document,relation,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,confidence
0,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",is_related,Bilateral breast,0,15,BodyPart,ultrasound,17,26,Imaging_Test,1.0
1,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",is_related,Bilateral breast,0,15,BodyPart,ovoid mass,78,87,ImagingFindings,0.99999714
2,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",is_related,ultrasound,17,26,Imaging_Test,ovoid mass,78,87,ImagingFindings,0.99956936
3,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",is_related,ovoid mass,78,87,ImagingFindings,0.5 x 0.5 x 0.4 cm,113,130,Measurements,1.0
4,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",is_related,mass,210,213,ImagingFindings,adjacent muscle,257,271,BodyPart,0.9976392
5,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",is_related,isoechoic echotexture,228,248,ImagingFindings,adjacent muscle,257,271,BodyPart,0.9999993


### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [10]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [11]:
import os

validation_file_name = "input.json"

validation_input_path = f"s3://{s3_bucket}/{model_name}/validation-input-json/batch"
validation_output_path = f"s3://{s3_bucket}/{model_name}/validation-output-json/batch"

input_dir = 'inputs/batch'
output_dir = 'outputs/batch'

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [12]:
import json

def write_and_upload_to_s3(json_input_data, file_name):

    json_data = json.dumps(json_input_data)

    with open(file_name, "w") as f:
        f.write(json_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input-json/batch/{os.path.basename(file_name)}",
        Body=(bytes(json_data.encode("UTF-8"))),
    )


In [13]:
# Define input JSON data for each validation file
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{input_dir}/{validation_file_name}")

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    accept="application/json",
)
transformer.transform(validation_input_path, content_type=content_type)
transformer.wait()

In [15]:
from urllib.parse import urlparse

def process_s3_output_and_save(validation_file_name, output_file_name):

    output_file_path = f"{output_dir}/{output_file_name}"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}/{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df_ner = pd.DataFrame(data['ner_predictions'])
    df_assertion = pd.DataFrame(data['assertion_predictions'])
    df_relation = pd.DataFrame(data['relation_predictions'])
    display(df_ner, df_assertion, df_relation)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [16]:
process_s3_output_and_save(validation_file_name, "out.out")

,document,ner_chunk,begin,end,ner_label,confidence
0,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",bilateral foot,69,82,BodyPart,0.615
1,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",pain,84,87,Symptom,0.7286
2,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",traumatic,165,173,Disease_Syndrome_Disorder,0.7773
3,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",arthritic,176,184,Disease_Syndrome_Disorder,0.6663
4,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",developmental abnormality,190,214,ImagingFindings,0.4231
5,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",Bilateral breast,0,15,BodyPart,0.945
6,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ultrasound,17,26,Imaging_Test,0.6734
7,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ovoid mass,78,87,ImagingFindings,0.6095
8,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",0.5 x 0.5 x 0.4,113,127,Measurements,0.9815801
9,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",cm,129,130,Units,0.9696


,document,ner_chunk,begin,end,ner_label,confidence,assertion
0,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",traumatic,165,173,Disease_Syndrome_Disorder,0.7773,Negative
1,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",arthritic,176,184,Disease_Syndrome_Disorder,0.6663,Negative
2,"Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality is discernible.",developmental abnormality,190,214,ImagingFindings,0.4231,Negative
3,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",ovoid mass,78,87,ImagingFindings,0.6095,Confirmed
4,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",mass,210,213,ImagingFindings,0.9711,Confirmed
5,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",isoechoic echotexture,228,248,ImagingFindings,0.80104995,Confirmed
6,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",internal color flow,294,312,ImagingFindings,0.47723332,Negative
7,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",benign fibrous tissue,334,354,ImagingFindings,0.5240666,Suspected
8,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",lipoma,361,366,Disease_Syndrome_Disorder,0.6081,Suspected


,document,relation,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,confidence
0,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",is_related,Bilateral breast,0,15,BodyPart,ultrasound,17,26,Imaging_Test,1.0
1,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",is_related,Bilateral breast,0,15,BodyPart,ovoid mass,78,87,ImagingFindings,0.99999714
2,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",is_related,ultrasound,17,26,Imaging_Test,ovoid mass,78,87,ImagingFindings,0.99956936
3,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",is_related,ovoid mass,78,87,ImagingFindings,0.5 x 0.5 x 0.4 cm,113,130,Measurements,1.0
4,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",is_related,mass,210,213,ImagingFindings,adjacent muscle,257,271,BodyPart,0.9976392
5,"Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.",is_related,isoechoic echotexture,228,248,ImagingFindings,adjacent muscle,257,271,BodyPart,0.9999993


In [17]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-explain-doc-clinical-radiology-pipel-2024-03-08-16-53-56-768


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

